In [ ]:
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
import torch
from create_database_functions import *

In [ ]:
file_path = "C:/Users/Iraklis Kolokas/Desktop/Thesis/Features"

Get features

In [ ]:
all_data = get_files(file_path)

Cosine Similarity

Averages between each session and for features from the same acoustic stimuli

In [ ]:
av_per_user, each_freq_av = get_averages(all_data)

Neural Network

In [ ]:
#Check if the system has a GPU
cuda = torch.cuda.is_available()
device = "cuda" if cuda else "cpu"
if cuda:
    torch.backends.cudnn.benchmark = True 

In [ ]:
features_1, stimuli_1, user_ids_1 = make_lists(all_data, 1)
features_2, stimuli_2, user_ids_2 = make_lists(all_data, 2)

In [ ]:
x_1, x_2, y_1, y_2, loader_1, loader_2 = make_databases(features_1, features_2, user_ids_1, user_ids_2)

In [ ]:
#Train the feature-only model
model1 = UserClassifier(x_1.shape[1], len(torch.unique(y_1)))
trained_model1 = train_classifier(model1, loader_1, n_epochs = 10, device)

model2 = UserClassifier(x_2.shape[1], len(torch.unique(y_2)))
trained_model2 = train_classifier(model2, loader_2, n_epochs = 10, device)

In [ ]:
#Add the stimuli labels to each feature with OneHotEncoder
stimuli_1 = np.hstack(stimuli_1)
stimuli_2 = np.hstack(stimuli_2)

stimuli_1 = stimuli_1.reshape(-1,1)
stimuli_2 = stimuli_2.reshape(-1,1)

encoder = OneHotEncoder(sparse_output = False)
stimuli_1 = encoder.fit_transform(stimuli_1)
stimuli_2 = encoder.fit_transform(stimuli_2)

#Add the encoded stimuli labels to the corresponding feature
features_1 = np.hstack([features_1, stimuli_1])
features_2 = np.hstack([features_2, stimuli_2])

In [ ]:
x_1, x_2, y_1, y_2, loader_1, loader_2 = make_databases(features_1, features_2, user_ids_1, user_ids_2)

In [ ]:
#Train the feature-label model
model3 = EEGUserClassifier(x_1.shape[1], len(torch.unique(y_1)))
trained_model3 = train_classifier(model3, loader_1, n_epochs = 10)

model4 = EEGUserClassifier(x_2.shape[1], len(torch.unique(y_2)))
trained_model4 = train_classifier(model4, loader_2, n_epochs = 10)

Save all the models

In [ ]:
torch.save(trained_model1.state_dict(), "C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_1.pth")
torch.save(trained_model2.state_dict(), "C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_2.pth")
torch.save(trained_model3.state_dict(), "C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_w_labels_1.pth")
torch.save(trained_model4.state_dict(), "C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_w_labels_2.pth")

Save averages

In [ ]:
#Save as pickle file to preserve dictionary structure
with open("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/each_freq_av.pkl" "wb") as f:
    pickle.dump(each_freq_av, f)
    
with open("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/av_per_user.pkl" "wb") as f:
    pickle.dump(av_per_user, f)